In [1]:

import pandas as pd
import numpy as np

# Dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc
from flask import Flask, jsonify, render_template, request
import csv
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float



In [2]:
crime_record = pd.read_csv("DataSets/Chicago_Crimes_2012_to_2017_v2.csv", index_col=False)
crime_record.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,3,10508693,HZ250496,05/03/2016 11:40:00 PM,013XX S SAWYER AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,...,24.0,29.0,08B,1154907.0,1893681.0,2016,05/10/2016 03:56:50 PM,41.864073,-87.706819,"(41.864073157, -87.706818608)"
1,89,10508695,HZ250409,05/03/2016 09:40:00 PM,061XX S DREXEL AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,...,20.0,42.0,08B,1183066.0,1864330.0,2016,05/10/2016 03:56:50 PM,41.782922,-87.604363,"(41.782921527, -87.60436317)"
2,197,10508697,HZ250503,05/03/2016 11:31:00 PM,053XX W CHICAGO AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,...,37.0,25.0,24,1140789.0,1904819.0,2016,05/10/2016 03:56:50 PM,41.894908,-87.758372,"(41.894908283, -87.758371958)"
3,673,10508698,HZ250424,05/03/2016 10:10:00 PM,049XX W FULTON ST,0460,BATTERY,SIMPLE,SIDEWALK,False,...,28.0,25.0,08B,1143223.0,1901475.0,2016,05/10/2016 03:56:50 PM,41.885687,-87.749516,"(41.885686845, -87.749515983)"
4,911,10508699,HZ250455,05/03/2016 10:00:00 PM,003XX N LOTUS AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,...,28.0,25.0,06,1139890.0,1901675.0,2016,05/10/2016 03:56:50 PM,41.886297,-87.761751,"(41.886297242, -87.761750709)"


In [3]:

crime_record = crime_record[["Year", "Date", "ID", "Case Number", "Description", "Arrest", "Ward", "Primary Type", "Location", "Block", "District" ]]
crime_record.head()

,Year,Date,ID,Case Number,Description,Arrest,Ward,Primary Type,Location,Block,District
0,2016,05/03/2016 11:40:00 PM,10508693,HZ250496,DOMESTIC BATTERY SIMPLE,True,24.0,BATTERY,"(41.864073157, -87.706818608)",013XX S SAWYER AVE,10.0
1,2016,05/03/2016 09:40:00 PM,10508695,HZ250409,DOMESTIC BATTERY SIMPLE,False,20.0,BATTERY,"(41.782921527, -87.60436317)",061XX S DREXEL AVE,3.0
2,2016,05/03/2016 11:31:00 PM,10508697,HZ250503,RECKLESS CONDUCT,False,37.0,PUBLIC PEACE VIOLATION,"(41.894908283, -87.758371958)",053XX W CHICAGO AVE,15.0
3,2016,05/03/2016 10:10:00 PM,10508698,HZ250424,SIMPLE,False,28.0,BATTERY,"(41.885686845, -87.749515983)",049XX W FULTON ST,15.0
4,2016,05/03/2016 10:00:00 PM,10508699,HZ250455,$500 AND UNDER,False,28.0,THEFT,"(41.886297242, -87.761750709)",003XX N LOTUS AVE,15.0


In [4]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
  
# Database setup
engine = create_engine("sqlite:///DataSets/chicago_crime33.sqlite")
# Create our session (link) from Python to the DB
session = Session(engine)

In [5]:
conn = engine.connect()
conn.execute('select Year from crime_record limit 4;').fetchall()

[(2016,), (2016,), (2016,), (2016,)]

In [15]:
Base = declarative_base()

class CrimeRecord (Base):
    __tablename__ = "crime_record"
    
    id_x = Column(Integer, primary_key=True)
    
    CaseNumber = Column(String)
    Year = Column(Integer)    
    PrimaryType = Column(Text)
    Description = Column(Text)
    Arrest = Column(Text)
    District = Column(Integer)
    Location = Column(Integer)
    Block = Column(String)
    
          
       #Date = Column(Integer)

 
    def __init__(self, CaseNumber, Year, PrimaryType, Description, Arrest, District, Location, Block):
        self.CaseNumber = CaseNumber
        self.Year = Year 
        self.Primary = PrimaryType
        self.Description = Description
        self.Arrest = Arrest
        self.District = District
        self.Location = Location
        self.Block = Block
            #return f"id={self.id}, name={self.name}"

In [ ]:
# Use Pandas to perform the sql query
stmt = session.query(CrimeRecord.Year).distinct()
df = pd.read_sql_query(stmt, session.bind)

# Return a list of the column years (sample years)
return jsonify(list(df['Year']))


In [ ]:
# # Use Pandas to perform the sql query
# stmt = session.query(CrimeRecord.Year)
# print(stmt)

# df = pd.read_sql_query(stmt, session.bind)

# # Return a list of the column years (sample years)

# print(df)

In [ ]:
x = session.query(CrimeRecord).limit(1)



In [7]:
Base.metadata.create_all(engine)

In [8]:
y =0
for Record in crime_record.to_dict(orient='records'):
    
    #print(Record)
    x = CrimeRecord(Record['Case Number'], Record['Year'], Record['Primary Type'], Record['Description'], Record['Arrest'], Record['District'], Record['Location'], Record['Block'])
    
    session.add(x) 
    session.commit()
    y += 1
    
    if y > 10:
        break
    
    
    

In [25]:
# Create an empty list for ID column
Description = []

# Query results from crime_record table
results = session.query(CrimeRecord.Description)

# # Loop through the query results and append the list with sample ids
for result in results:
    Description.append(result[0])
Description

['DOMESTIC BATTERY SIMPLE',
 'DOMESTIC BATTERY SIMPLE',
 'RECKLESS CONDUCT',
 'SIMPLE',
 '$500 AND UNDER',
 'AGGRAVATED: HANDGUN',
 'SIMPLE',
 'SIMPLE',
 'SIMPLE',
 'DOMESTIC BATTERY SIMPLE',
 'UNLAWFUL POSS OF HANDGUN',
 'DOMESTIC BATTERY SIMPLE',
 'FROM BUILDING',
 '$500 AND UNDER',
 'OVER $500',
 'DOMESTIC BATTERY SIMPLE',
 'OVER $500',
 'ARMED: OTHER DANGEROUS WEAPON',
 'AUTOMOBILE',
 'DOMESTIC BATTERY SIMPLE',
 'SIMPLE',
 'SIMPLE',
 'OVER $500',
 '$500 AND UNDER',
 'OTHER VEHICLE OFFENSE',
 '$500 AND UNDER',
 '$500 AND UNDER',
 'FRAUD OR CONFIDENCE GAME',
 'FINANCIAL IDENTITY THEFT OVER $ 300',
 'AUTOMOBILE',
 'AUTOMOBILE',
 'SIMPLE',
 '$500 AND UNDER',
 'OVER $500',
 'CRIMINAL DEFACEMENT',
 'FINANCIAL IDENTITY THEFT OVER $ 300',
 '$500 AND UNDER',
 'TO PROPERTY',
 'OVER $500',
 'DOMESTIC BATTERY SIMPLE',
 '$500 AND UNDER',
 'AUTOMOBILE',
 'CRIMINAL DEFACEMENT',
 'HARASSMENT BY TELEPHONE',
 '$500 AND UNDER',
 'TO PROPERTY',
 'RECKLESS FIREARM DISCHARGE',
 'AUTOMOBILE',
 'TELEPHONE

In [24]:
CaseNumber = []
reseults = session.query(CrimeRecord.CaseNumber)

for x in results:
    CaseNumber.append(result[0])

CaseNumber

['1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1'

In [ ]:
#print(data[:5])

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

Records = Base.classes.Records
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
#################################################
# Flask Setup
#################################################
app = Flask(__name__)
#################################################
# Flask Routes
#################################################
# Returns the dashboard homepage
@app.route("/")
def index():
    return render_template("index.html")

In [ ]:
#################################################
# Returns a list of sample names
@app.route('/names')
def names():
    """Return a list of sample names."""

    # Use Pandas to perform the sql query
    stmt = session.query(Samples).statement
    df = pd.read_sql_query(stmt, session.bind)
    df.set_index('otu_id', inplace=True)

    # Return a list of the column names (sample names)
    return jsonify(list(df.columns))


In [ ]:
#################################################
# Returns a list of OTU descriptions 
@app.route('/otu')
def otu():
    """Return a list of OTU descriptions."""
    results = session.query(OTU.lowest_taxonomic_unit_found).all()

    # Use numpy ravel to extract list of tuples into a list of OTU descriptions
    otu_list = list(np.ravel(results))
    return jsonify(otu_list)

In [ ]:
# Save the reference to the `customers` table as a variable called `table`
table = sqlalchemy.Table("chicago_crime", metadata, autoload=True)

In [ ]:
# Save the reference to the `customers` table as a variable called `table`
table = sqlalchemy.Table('chicago_crime', metadata, autoload=True)

In [ ]:
columns = [m.key for m in table.columns]
columns

In [ ]:
#################################################
# Returns a json dictionary of sample metadata 
@app.route('/metadata/<sample>')
def sample_metadata(sample):
    """Return the MetaData for a given sample."""
    sel = [Samples_Metadata.SAMPLEID, Samples_Metadata.ETHNICITY,
           Samples_Metadata.GENDER, Samples_Metadata.AGE,
           Samples_Metadata.LOCATION, Samples_Metadata.BBTYPE]

    # sample[3:] strips the `BB_` prefix from the sample name to match
    # the numeric value of `SAMPLEID` from the database
    results = session.query(*sel).\
        filter(Samples_Metadata.SAMPLEID == sample[3:]).all()

    # Create a dictionary entry for each row of metadata information
    sample_metadata = {}
    for result in results:
        sample_metadata['SAMPLEID'] = result[0]
        sample_metadata['ETHNICITY'] = result[1]
        sample_metadata['GENDER'] = result[2]
        sample_metadata['AGE'] = result[3]
        sample_metadata['LOCATION'] = result[4]
        sample_metadata['BBTYPE'] = result[5]

    return jsonify(sample_metadata)
